In [2]:
import kagglehub
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score, roc_auc_score
from sklearn.model_selection import train_test_split

# 1. Load dataset with memory efficiency
dataset_path = kagglehub.dataset_download("aitude/aptos-augmented-images")
print("Dataset Path:", dataset_path)

all_images = []
all_labels = []

# Walk through folders and only load a subset first to prevent OOM
max_images_per_class = 500  # limit per class to avoid memory crash

for root, dirs, files in os.walk(dataset_path):
    label_name = os.path.basename(root)
    if label_name == os.path.basename(dataset_path):
        continue
    count = 0
    for file in files:
        if file.lower().endswith((".jpg", ".png", ".jpeg")):
            if count >= max_images_per_class:
                break
            img_path = os.path.join(root, file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (128,128))  # reduce resolution to save memory
            img = img.astype(np.float32) / 255.0
            all_images.append(img)
            all_labels.append(label_name)
            count += 1
    if count > 0:
        print(f"Loaded {count} images for label {label_name}")

X = np.array(all_images)
label_to_idx = {label: idx for idx, label in enumerate(sorted(set(all_labels)))}
y = np.array([label_to_idx[l] for l in all_labels])

print("Dataset loaded:", X.shape, y.shape, "Unique labels:", np.unique(y))

if X.shape[0] == 0:
    raise RuntimeError("No images loaded. Check dataset structure and image formats.")

# 2. Build lightweight CNN feature extractor
def build_cnn():
    model = models.Sequential([
        layers.Conv2D(16, (3,3), activation='relu', input_shape=(128,128,3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),

        layers.Conv2D(32, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),

        layers.Conv2D(64, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),

        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu')
    ])
    return model

cnn_model = build_cnn()

# Extract features in batches to reduce memory usage
print("Extracting features with CNN...")
features = cnn_model.predict(tf.data.Dataset.from_tensor_slices(X).batch(32), verbose=1)

# 3. Ensemble SVM classifier
svm1 = SVC(kernel='linear', probability=True)
svm2 = SVC(kernel='rbf', probability=True)
svm3 = SVC(kernel='poly', degree=3, probability=True)

ensemble_svm = VotingClassifier(estimators=[('svm1', svm1), ('svm2', svm2), ('svm3', svm3)], voting='soft')

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42, stratify=y)

ensemble_svm.fit(X_train, y_train)

# Predictions
y_pred = ensemble_svm.predict(X_test)
y_prob = ensemble_svm.predict_proba(X_test)

# 4. Evaluation
acc = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='macro')
precision = precision_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
kappa = cohen_kappa_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob, multi_class='ovr')

print("Accuracy:", acc)
print("Recall (Sensitivity):", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("Cohen Kappa:", kappa)
print("AUC:", auc)


Dataset Path: /kaggle/input/aptos-augmented-images
Loaded 500 images for label 2
Loaded 500 images for label 0
Loaded 500 images for label 3
Loaded 500 images for label 1
Loaded 500 images for label 4
Loaded 500 images for label 2
Loaded 500 images for label 0
Loaded 500 images for label 3
Loaded 500 images for label 1
Loaded 500 images for label 4
Dataset loaded: (5000, 128, 128, 3) (5000,) Unique labels: [0 1 2 3 4]
Extracting features with CNN...


I0000 00:00:1757688064.567732     771 service.cc:148] XLA service 0x7a7fc4003f70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1757688064.569012     771 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1757688064.569034     771 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


 38/157 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

I0000 00:00:1757688065.453730     771 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
Accuracy: 0.413
Recall (Sensitivity): 0.413
Precision: 0.3936715107699323
F1 Score: 0.3910225750691553
Cohen Kappa: 0.26625
AUC: 0.7291274999999999


In [3]:
import kagglehub
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score, roc_auc_score
from sklearn.model_selection import train_test_split
from skimage.filters import threshold_otsu

# 1. Load dataset with memory efficiency
dataset_path = kagglehub.dataset_download("aitude/aptos-augmented-images")
print("Dataset Path:", dataset_path)

all_images = []
all_labels = []

# Walk through folders and only load a subset first to prevent OOM
max_images_per_class = 500  # limit per class to avoid memory crash

# --- Fuzzy entropy preprocessing helper ---
def fuzzy_entropy_preprocess(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Normalize
    gray = gray.astype(np.float32) / 255.0
    # Compute global threshold using Otsu (approximation for fuzzy entropy thresholding)
    thresh = threshold_otsu(gray)
    mask = (gray > thresh).astype(np.float32)
    # Apply mask back to 3-channel image
    proc = img * mask[..., None]
    return proc

for root, dirs, files in os.walk(dataset_path):
    label_name = os.path.basename(root)
    if label_name == os.path.basename(dataset_path):
        continue
    count = 0
    for file in files:
        if file.lower().endswith((".jpg", ".png", ".jpeg")):
            if count >= max_images_per_class:
                break
            img_path = os.path.join(root, file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (128,128))
            # Apply fuzzy entropy preprocessing
            img = fuzzy_entropy_preprocess(img)
            img = img.astype(np.float32) / 255.0
            all_images.append(img)
            all_labels.append(label_name)
            count += 1
    if count > 0:
        print(f"Loaded {count} images for label {label_name}")

X = np.array(all_images)
label_to_idx = {label: idx for idx, label in enumerate(sorted(set(all_labels)))}
y = np.array([label_to_idx[l] for l in all_labels])

print("Dataset loaded:", X.shape, y.shape, "Unique labels:", np.unique(y))

if X.shape[0] == 0:
    raise RuntimeError("No images loaded. Check dataset structure and image formats.")

# 2. Build lightweight CNN feature extractor
def build_cnn():
    model = models.Sequential([
        layers.Conv2D(16, (3,3), activation='relu', input_shape=(128,128,3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),

        layers.Conv2D(32, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),

        layers.Conv2D(64, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),

        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu')
    ])
    return model

cnn_model = build_cnn()

# Extract features in batches to reduce memory usage
print("Extracting features with CNN...")
features = cnn_model.predict(tf.data.Dataset.from_tensor_slices(X).batch(32), verbose=1)

# 3. Ensemble SVM classifier
svm1 = SVC(kernel='linear', probability=True)
svm2 = SVC(kernel='rbf', probability=True)
svm3 = SVC(kernel='poly', degree=3, probability=True)

ensemble_svm = VotingClassifier(estimators=[('svm1', svm1), ('svm2', svm2), ('svm3', svm3)], voting='soft')

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42, stratify=y)

ensemble_svm.fit(X_train, y_train)

# Predictions
y_pred = ensemble_svm.predict(X_test)
y_prob = ensemble_svm.predict_proba(X_test)

# 4. Evaluation
acc = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='macro')
precision = precision_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
kappa = cohen_kappa_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob, multi_class='ovr')

print("Accuracy:", acc)
print("Recall (Sensitivity):", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("Cohen Kappa:", kappa)
print("AUC:", auc)

Dataset Path: /kaggle/input/aptos-augmented-images
Loaded 500 images for label 2
Loaded 500 images for label 0
Loaded 500 images for label 3
Loaded 500 images for label 1
Loaded 500 images for label 4
Loaded 500 images for label 2
Loaded 500 images for label 0
Loaded 500 images for label 3
Loaded 500 images for label 1
Loaded 500 images for label 4
Dataset loaded: (5000, 128, 128, 3) (5000,) Unique labels: [0 1 2 3 4]
Extracting features with CNN...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
Accuracy: 0.402
Recall (Sensitivity): 0.40199999999999997
Precision: 0.36004039504423074
F1 Score: 0.34682958443332057
Cohen Kappa: 0.25249999999999995
AUC: 0.70919125
